In [1]:
HOPSWORKS_PROJECT_NAME = 'NYCTaxiDemand'

In [2]:
import sys
import os
from dotenv import load_dotenv

# Get the absolute path to the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
src_path = os.path.join(project_root, 'src')
sys.path.append(project_root)  # Add the project root to the Python path

# Now you can import your custom modules
from src.paths import PARENT_DIR

load_dotenv(PARENT_DIR / '.env')

HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']

In [3]:
import sys
import os
from datetime import datetime
import pandas as pd

# Get the absolute path to the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
src_path = os.path.join(project_root, 'src')
sys.path.append(project_root)  # Add the project root to the Python path

from src.data import load_raw_data

from_year = 2022
to_year = datetime.now().year
print(f'Downloading raw data from {from_year} to {to_year}')

rides = pd.DataFrame()
for year in range(from_year, to_year+1):
    # download data for whole year
    rides_one_year = load_raw_data(year)

    # append rows
    rides = pd.concat([rides, rides_one_year])

Checking for file: /Users/eugene/Github/taxi_demand_predictor/data/raw/rides_2022-01.parquet
File 2022-01 was already in local storage: /Users/eugene/Github/taxi_demand_predictor/data/raw/rides_2022-01.parquet
Files in RAW_DATA_DIR: ['rides_2024-01.parquet', 'rides_2023-03.parquet', 'rides_2022-05.parquet', 'rides_2023-02.parquet', 'rides_2023-12.parquet', 'rides_2022-04.parquet', 'rides_2023-10.parquet', 'rides_2022-06.parquet', 'rides_2023-09.parquet', 'rides_2024-02.parquet', 'rides_2023-08.parquet', 'rides_2024-03.parquet', 'rides_2023-01.parquet', 'rides_2023-11.parquet', 'rides_2022-07.parquet', 'rides_2022-12.parquet', 'rides_2022-02.parquet', 'rides_2023-04.parquet', 'rides_2022-03.parquet', 'rides_2023-05.parquet', 'rides_2022-08.parquet', 'rides_2022-11.parquet', 'rides_2022-01.parquet', 'rides_2023-07.parquet', 'rides_2022-10.parquet', 'rides_2023-06.parquet', 'rides_2022-09.parquet', 'rides_2024-04.parquet']
Successfully loaded file: /Users/eugene/Github/taxi_demand_predict

In [4]:
print(f'{len(rides)=}')

len(rides)=91033132


In [5]:
import sys
import os

# Get the absolute path to the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
src_path = os.path.join(project_root, 'src')
sys.path.append(project_root)  # Add the project root to the Python path

from src.data import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 263/263 [00:20<00:00, 12.64it/s]


In [6]:
ts_data

,pickup_hour,rides,pickup_location_id
0,2022-01-01 00:00:00,11,4
1,2022-01-01 01:00:00,15,4
2,2022-01-01 02:00:00,26,4
3,2022-01-01 03:00:00,8,4
4,2022-01-01 04:00:00,9,4
...,...,...,...
5371507,2024-04-30 19:00:00,0,110
5371508,2024-04-30 20:00:00,0,110
5371509,2024-04-30 21:00:00,0,110
5371510,2024-04-30 22:00:00,0,110


In [7]:
import hopsworks

In [8]:
project = hopsworks.login(
    project=HOPSWORKS_PROJECT_NAME,
    api_key_value=HOPSWORKS_API_KEY
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/837781


In [9]:
feature_store = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [10]:
FEATURE_GROUP_NAME = 'time_series_hourly_feature_group'
FEATURE_GROUP_VERSION = 1

In [11]:
feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description='Time-series data at hourly frequency',
    primary_key=['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour'
)

In [13]:
feature_group.insert(ts_data,
                     write_options={'wait_for_job':False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/837781/fs/833604/fg/955745


Uploading Dataframe: 0.00% |          | Rows 0/5371512 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/837781/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f75dc7ddaf0>, None)